<a href="https://colab.research.google.com/github/PoloVlzq/Clase/blob/main/Invenory%20con%20Simpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install simpy

import simpy
import numpy as np
def warehouse_run(env, order_cutoff, order_target):
  global inventory , balance, num_ordered
  inventory =order_target
  balance=0.0
  num_ordered=0

  while True:
    interarrival=generate_interarrival()
    yield env.timeout(interarrival)
    balance-=inventory*2*interarrival
    demand=generate_demand()
    if demand < inventory:
      balance+=100*demand
      inventory-=demand
      print('{:.f}sold{}'.format(env.now,demand))
    else:
      balance += 100*inventory
      inventory = 0
      print('{:.2f}sold{}(out of stock)'.format(env.now,inventory))
    if inventory < order_cutoff and num_ordered == 0:
      env.process(handle_order(env,order_target))

def handle_order(env,order_target):
  global inventory, balance, num_ordered

  num_ordered = order_target - inventory
  print('{:.2f}placed order for{}'.format(env.now, num_ordered))
  balance -= 50*num_ordered
  yield env.timeout(2.0)
  inventory += num_ordered
  num_ordered = 0
  print('{:.2f}received order, {}in inventory'.format(env.now, inventory))

def generate_interarrival():
  return np.random.exponential(1./5)

def generate_demand():
  return np.random.randint(1,5)

obs_time = []
inventory_level = []

def observe(env):
  global inventory

  while True:
    obs_time.append(env.now)
    inventory_level.append(inventory)
    yield env.timeout(0.1)

np.random.seed(2)

env=simpy.Environment()
env.process(warehouse_run(env,10,30))
env.process(observe(env))

env.run(until=5.0)

import matplotlib.pyplot as plt

plt.figure()
plt.step(obs_time, inventory_level, where='post')
plt.xlabel('Simulation time(days)')
plt.ylabel('Inventory level')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ValueError: ignored